# Exploratory Data Analysis en el mercado cripto.

In [1]:
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set()

%matplotlib inline

# Tamaño de las figuras
sns.set(rc={"figure.figsize": (18,9)})

# Autocompletar más rápido
%config Completer.use_jedi = False

# Mostrar solo dos decimales en pandas
pd.options.display.float_format = "{:.2f}".format

sns.set_style('whitegrid')

### Lo primero que vamos a hacer es conjuntar los datos de binance con lo que podamos de coingecko.

In [18]:
cg_btc = pd.read_csv('CoinGecko/data/btc.csv')
cg_btc['Fecha'] = pd.to_datetime(cg_btc.Fecha)
# cg_ada = pd.read_csv('CoinGecko/data/ada.csv')
# cg_bnb = pd.read_csv('CoinGecko/data/bnb.csv')
# cg_doge = pd.read_csv('CoinGecko/data/doge.csv')
# cg_dot = pd.read_csv('CoinGecko/data/dot.csv')
# cg_eth = pd.read_csv('CoinGecko/data/eth.csv')
# cg_ltc = pd.read_csv('CoinGecko/data/ltc.csv')
# cg_matic = pd.read_csv('CoinGecko/data/matic.csv')
# cg_shib = pd.read_csv('CoinGecko/data/shib.csv')
# cg_sol = pd.read_csv('CoinGecko/data/sol.csv')
# cg_trx = pd.read_csv('CoinGecko/data/trx.csv')
# cg_xrp = pd.read_csv('CoinGecko/data/xrp.csv')

bn_btc = pd.read_csv('Binance/data/by1d/btcusdt.csv')
bn_btc['Fecha'] = pd.to_datetime(bn_btc.Fecha)
# bn_ada = pd.read_csv('Binance/data/by1d/adausdt.csv')
# bn_bnb = pd.read_csv('Binance/data/by1d/bnbusdt.csv')
# bn_doge = pd.read_csv('Binance/data/by1d/dogeusdt.csv')
# bn_dot = pd.read_csv('Binance/data/by1d/dotusdt.csv')
# bn_eth = pd.read_csv('Binance/data/by1d/ethusdt.csv')
# bn_ltc = pd.read_csv('Binance/data/by1d/ltcusdt.csv')
# bn_matic = pd.read_csv('Binance/data/by1d/maticusdt.csv')
# bn_shib = pd.read_csv('Binance/data/by1d/shibusdt.csv')
# bn_sol = pd.read_csv('Binance/data/by1d/solusdt.csv')
# bn_trx = pd.read_csv('Binance/data/by1d/trxusdt.csv')
# bn_xrp = pd.read_csv('Binance/data/by1d/xrpusdt.csv')

bn_btc

,Fecha,Abre,Alto,Bajo,Cierra,Volumen
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.15
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.89
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.31
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.08
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.74
...,...,...,...,...,...,...
2188,2023-08-14,29303.85,29695.32,29102.45,29430.93,31443.12
2189,2023-08-15,29430.92,29499.26,29059.60,29200.00,27503.88
2190,2023-08-16,29200.01,29259.85,28723.08,28730.51,32996.70
2191,2023-08-17,28730.51,28783.48,25166.00,26623.41,100271.54


In [19]:
cg_btc

,Fecha,Precios,Volumen
0,2013-04-29 00:00:00,141.96,0.00
1,2013-04-30 00:00:00,135.30,0.00
2,2013-05-01 00:00:00,117.00,0.00
3,2013-05-02 00:00:00,103.43,0.00
4,2013-05-03 00:00:00,91.01,0.00
...,...,...,...
3758,2023-08-15 00:00:00,29400.59,12408355477.11
3759,2023-08-16 00:00:00,29170.49,11523395158.00
3760,2023-08-17 00:00:00,28754.20,14809979642.37
3761,2023-08-18 00:00:00,26501.59,29965585683.42


In [27]:
# pd.to_datetime(cg_btc.Fecha)
# type(cg_btc.Fecha[0]), type(bn_btc.Fecha[0])
cg_btc.Fecha[-2:-1].item() == bn_btc.Fecha[-1:].item()

True

In [36]:
df = bn_btc[['Fecha','Cierra','Volumen']]
df.columns = ['Fecha','Precios','Volumen']
df = df.merge(cg_btc,suffixes=('_bn','_cg'),how='left',on='Fecha')
# df['Volumen_cg'] = df.Volumen_cg / 1000000000
df

,Fecha,Precios_bn,Volumen_bn,Precios_cg,Volumen_cg
0,2017-08-17,4285.08,795.15,4286.92,1774245121.03
1,2017-08-18,4108.37,1199.89,4097.69,1515069151.86
2,2017-08-19,4139.98,381.31,4115.21,1710080500.02
3,2017-08-20,4086.29,467.08,4001.23,551798951.97
4,2017-08-21,4016.00,691.74,3935.23,712634899.40
...,...,...,...,...,...
2188,2023-08-14,29430.93,31443.12,29284.97,4755220448.63
2189,2023-08-15,29200.00,27503.88,29400.59,12408355477.11
2190,2023-08-16,28730.51,32996.70,29170.49,11523395158.00
2191,2023-08-17,26623.41,100271.54,28754.20,14809979642.37
